# Consultas 

## Líbrerías a ocupar 

In [3]:
import numpy as np
import pandas as pd
from datetime import datetime

In [4]:
comunas = pd.read_csv("data/comunas.csv")
plebiscitoAR = pd.read_csv("data/Plebiscito Apruebo-Rechazo.csv", sep=";", decimal=",")

plebiscitoC = pd.read_csv("data/Plebiscito convención.csv", sep=";", decimal=",")
presidencial1ra = pd.read_csv("data/resultadosPresidencial1era.csv")
presidencial2da = pd.read_csv("data/resultadosPresidencial2da.csv")

ingresos = pd.read_csv("data/porcentajePobreza.csv")
candidatos1era = list(presidencial1ra.columns)[1:]
candidatos2da = list(presidencial2da.columns)[1:]
piniera1era = candidatos1era.index('Sebastian Piniera Echenique')
piniera2da = candidatos2da.index('Sebastian Piniera Echenique')

## Cálculo del peso de votar por piñera en el apruebo

In [5]:
general = plebiscitoAR.join(ingresos.set_index("comuna"), on = "Comuna")
general = general.join(presidencial2da.set_index("comuna"), on="Comuna")
general.head()


,Comuna,Apruebo (1),Rechazo (0),Ganador,Total,Porcentaje Apruebo,Porcentaje Rechazo,porcentajePobreza,categoria,Sebastian Piniera Echenique,Alejandro Guillier Alvarez,Votos Nulos,Votos Blancos,ganador
0,Algarrobo,4932,2118,1,7050,0.700,0.300,0.03275,0,4138,2519,34,5,0
1,Alhue,2323,309,1,2632,0.883,0.117,0.06713,0,1028,1425,13,5,1
2,Alto Biobio,1269,424,1,1693,0.750,0.250,0.39734,3,1243,701,47,17,0
3,Alto del Carmen,1559,241,1,1800,0.866,0.134,0.15273,1,1066,847,9,7,0
4,Alto Hospicio,21589,4534,1,26123,0.826,0.174,0.07755,0,9969,7466,115,32,0


In [6]:
## probabilidad categoria ingreso

values = general["categoria"].value_counts()
categ = {}
for i in values.index:
    categ[i] = values[i]/comunas.shape[0]
categ

{1: 0.4289855072463768,
 0: 0.4144927536231884,
 2: 0.1246376811594203,
 3: 0.028985507246376812,
 4: 0.002898550724637681}

In [7]:
## probabilidad apruebo que voto por piñera 
init = datetime.now()
apruebo = 1
aprobPiniera = 0
for i in values.index:
    match = general.loc[(general["Ganador"]==apruebo) & (general["ganador"]==piniera2da) & (general["categoria"]==i)]
    total = general.loc[(general["ganador"]==piniera2da) & (general["categoria"]==i)]
    if(not total.shape[0]):
        continue
    aprobPiniera += (match.shape[0]/total.shape[0]) * categ[i]
    
aprobNoPiniera = 0
## probabilidad apruebo que no voto por piñera
for i in values.index:
    match = general.loc[(general["Ganador"]==apruebo) & (general["ganador"]!=piniera2da) & (general["categoria"]==i)]
    total = general.loc[(general["ganador"]!=piniera2da) & (general["categoria"]==i)]
    if(not total.shape[0]):
        continue
    aprobNoPiniera += (match.shape[0]/total.shape[0]) * categ[i]

## probabilidad rechazo que voto por piñera 
apruebo = 0
rechazoPiniera = 0
for i in values.index:
    match = general.loc[(general["Ganador"]==apruebo) & (general["ganador"]==piniera2da) & (general["categoria"]==i)]
    total = general.loc[(general["ganador"]==piniera2da) & (general["categoria"]==i)]
    if(not total.shape[0]):
        continue
    rechazoPiniera += (match.shape[0]/total.shape[0]) * categ[i]

## probabilidad rechazo que no voto por piñera 
apruebo = 0
rechazoNoPiniera = 0
for i in values.index:
    match = general.loc[(general["Ganador"]==apruebo) & (general["ganador"]!=piniera2da) & (general["categoria"]==i)]
    total = general.loc[(general["ganador"]!=piniera2da) & (general["categoria"]==i)]
    if(not total.shape[0]):
        continue
    rechazoNoPiniera += (match.shape[0]/total.shape[0]) * categ[i]

meanPiniera = 1*aprobPiniera + 0*rechazoPiniera
meanNoPiniera = 1*aprobNoPiniera + 0*rechazoNoPiniera
print(f"Peso voto por Piñera: {meanPiniera - meanNoPiniera}")
meanPiniera = 1*aprobPiniera - 1*rechazoPiniera
meanNoPiniera = 1*aprobNoPiniera - 1*rechazoNoPiniera
print(f"Peso voto por Piñera (con -1): {meanPiniera - meanNoPiniera}")
final = datetime.now()
print(f"Tiempo de ejecución: {final-init}")

Peso voto por Piñera: -0.0049511739736753935
Peso voto por Piñera (con -1): -0.012800898671988414
Tiempo de ejecución: 0:00:00.062997


## Cálculo del peso de votar por piñera en la convención Constitucional

In [8]:
general = plebiscitoC.join(ingresos.set_index("comuna"), on = "Comuna")
general = general.join(presidencial2da.set_index("comuna"), on="Comuna")
general.head()

,Comuna,Convencion Mixta (1),Convencion Constitucional (0),Columna1,Total,Porcentaje CC,Porcentaje CM,porcentajePobreza,categoria,Sebastian Piniera Echenique,Alejandro Guillier Alvarez,Votos Nulos,Votos Blancos,ganador
0,Algarrobo,1856,4754,0,6610,0.281,0.719,0.03275,0,4138,2519,34,5,0
1,Alhue,406,2158,0,2564,0.158,0.842,0.06713,0,1028,1425,13,5,1
2,Alto Biobio,424,1131,0,1555,0.273,0.727,0.39734,3,1243,701,47,17,0
3,Alto del Carmen,278,1465,0,1743,0.159,0.841,0.15273,1,1066,847,9,7,0
4,Alto Hospicio,4663,20247,0,24910,0.187,0.813,0.07755,0,9969,7466,115,32,0


In [9]:
##probabilidad mixta que voto por piñera
init = datetime.now()
mixta = 1
mixtaPiniera = 0
for i in values.index:
    match = general.loc[(general["Columna1"]==mixta) & (general["ganador"]==piniera2da) & (general["categoria"]==i)]
    total = general.loc[(general["ganador"]==piniera2da) & (general["categoria"]==i)]
    if(not total.shape[0]):
        continue
    mixtaPiniera += (match.shape[0]/total.shape[0]) * categ[i]

mixtaNoPiniera = 0
## probabilidad mixta que no voto por piñera
for i in values.index:
    match = general.loc[(general["Columna1"]==mixta) & (general["ganador"]!=piniera2da) & (general["categoria"]==i)]
    total = general.loc[(general["ganador"]!=piniera2da) & (general["categoria"]==i)]
    if(not total.shape[0]):
        continue
    mixtaNoPiniera += (match.shape[0]/total.shape[0]) * categ[i]

## probabilidad no mixta que voto por piñera 
mixta = 0
noMixtaPiniera = 0
for i in values.index:
    match = general.loc[(general["Columna1"]==mixta) & (general["ganador"]==piniera2da) & (general["categoria"]==i)]
    total = general.loc[(general["ganador"]==piniera2da) & (general["categoria"]==i)]
    if(not total.shape[0]):
        continue
    noMixtaPiniera += (match.shape[0]/total.shape[0]) * categ[i]

## probabilidad no mixta que no voto por piñera 

noMixtaNoPiniera = 0
for i in values.index:
    match = general.loc[(general["Columna1"]==mixta) & (general["ganador"]!=piniera2da) & (general["categoria"]==i)]
    total = general.loc[(general["ganador"]!=piniera2da) & (general["categoria"]==i)]
    if(not total.shape[0]):
        continue
    noMixtaNoPiniera += (match.shape[0]/total.shape[0]) * categ[i]

meanPiniera = 1*mixtaPiniera + 0*noMixtaPiniera
meanNoPiniera = 1*mixtaNoPiniera + 0*noMixtaNoPiniera
print(f"Peso voto por Piñera: {meanPiniera - meanNoPiniera}")
meanPiniera = 1*mixtaPiniera - 1*noMixtaPiniera
meanNoPiniera = 1*mixtaNoPiniera - 1*noMixtaNoPiniera
print(f"Peso voto por Piñera (con -1): {meanPiniera - meanNoPiniera}")
final = datetime.now()
print(f"Tiempo de ejecución: {final-init}")

Peso voto por Piñera: 0.013516068052930055
Peso voto por Piñera (con -1): 0.024133585381222566
Tiempo de ejecución: 0:00:00.056998


## Cálculo del peso por votar piñera en primera vuelta

In [15]:
general = plebiscitoAR.join(ingresos.set_index("comuna"), on = "Comuna")
general = general.join(presidencial2da.set_index("comuna").add_suffix("2"), on="Comuna")
general = general.join(presidencial1ra.set_index("comuna"), on = "Comuna")

In [23]:
values = general["ganador"].value_counts()
ganador1 = {}
for i in values.index:
    ganador1[i] = values[i]/comunas.shape[0]
ganador1

{2: 0.8840579710144928,
 3: 0.10434782608695652,
 4: 0.008695652173913044,
 0: 0.002898550724637681}

In [17]:
values = general["ganador2"].value_counts()
ganador2 = {}
for i in values.index:
    ganador2[i] = values[i]/comunas.shape[0]
ganador2

{0: 0.7391304347826086, 1: 0.2608695652173913}

In [20]:

init = datetime.now()
ganador2Cond1 = {}
for i in ganador2:
    match = general.loc[(general["ganador2"]==i) & (general["ganador"]==piniera1era)]
    total = general.loc[general["ganador"]==piniera1era]
    if(not total.shape[0]):
        continue
    ganador2Cond1[i] = match.shape[0]/total.shape[0]


apruebo = 1
aprobPiniera = 0
for i in values.index:
    for j in ganador2:
        match = general.loc[(general["Ganador"]==apruebo) & (general["ganador"]==piniera1era) & (general["categoria"]==i) & (general["ganador2"]==j)]
        total = general.loc[(general["ganador"]==piniera1era) & (general["categoria"]==i) & (general["ganador2"]==j)]
        if(not total.shape[0]):
            continue
        aprobPiniera += (match.shape[0]/total.shape[0]) * categ[i] * ganador2[j] * ganador2Cond1[j]

aprobNoPiniera = 0
## probabilidad apruebo que no voto por piñera
for i in values.index:
    for j in ganador2:
        match = general.loc[(general["Ganador"]==apruebo) & (general["ganador"]!=piniera1era) & (general["categoria"]==i) & (general["ganador2"]==j)]
        total = general.loc[(general["ganador"]!=piniera1era) & (general["categoria"]==i) & (general["ganador2"]==j)]
        if(not total.shape[0]):
            continue
        aprobNoPiniera += (match.shape[0]/total.shape[0]) * categ[i] * ganador2[j] * ganador2Cond1[j]


## probabilidad rechazo que voto por piñera 
apruebo = 0
rechazoPiniera = 0
for i in values.index:
    for j in ganador2:
        match = general.loc[(general["Ganador"]==apruebo) & (general["ganador"]==piniera1era) & (general["categoria"]==i) & (general["ganador2"]==j)]
        total = general.loc[(general["ganador"]==piniera1era) & (general["categoria"]==i) & (general["ganador2"]==j)]
        if(not total.shape[0]):
            continue
        rechazoPiniera += (match.shape[0]/total.shape[0]) * categ[i] * ganador2[j] * ganador2Cond1[j]


## probabilidad rechazo que no voto por piñera 
apruebo = 0
rechazoNoPiniera = 0
for i in values.index:
    for j in ganador2:
        match = general.loc[(general["Ganador"]==apruebo) & (general["ganador"]!=piniera1era) & (general["categoria"]==i) & (general["ganador2"]==j)]
        total = general.loc[(general["ganador"]!=piniera1era) & (general["categoria"]==i) & (general["ganador2"]==j)]
        if(not total.shape[0]):
            continue
        rechazoPiniera += (match.shape[0]/total.shape[0]) * categ[i] * ganador2[j] * ganador2Cond1[j]

meanPiniera = 1*aprobPiniera + 0*rechazoPiniera
meanNoPiniera = 1*aprobNoPiniera + 0*rechazoNoPiniera
print(f"Peso voto por Piñera: {meanPiniera - meanNoPiniera}")
meanPiniera = 1*aprobPiniera - 1*rechazoPiniera
meanNoPiniera = 1*aprobNoPiniera - 1*rechazoNoPiniera
print(f"Peso voto por Piñera (con -1): {meanPiniera - meanNoPiniera}")
final = datetime.now()
print(f"Tiempo de ejecución: {final-init}")

Peso voto por Piñera: 0.2413194579457913
Peso voto por Piñera (con -1): 0.22651999251429972
Tiempo de ejecución: 0:00:00.038995


## Consulta peso de Piñera en 1era sin causalidad de la 2da vuelta

In [21]:
general = plebiscitoAR.join(ingresos.set_index("comuna"), on = "Comuna")
general = general.join(presidencial1ra.set_index("comuna"), on="Comuna")
general.head()

,Comuna,Apruebo (1),Rechazo (0),Ganador,Total,Porcentaje Apruebo,Porcentaje Rechazo,porcentajePobreza,categoria,Carolina Goic Boroevic,Jose Antonio Kast Rist,Sebastian Piniera Echenique,Alejandro Guillier Alvarez,Beatriz Sanchez Munioz,Marco Enriquez-ominami Gumucio,Eduardo Artes Brichetti,Alejandro Navarro Brain,Votos Nulos,Votos Blancos,ganador
0,Algarrobo,4932,2118,1,7050,0.700,0.300,0.03275,0,405,338,2974,1170,1099,247,24,12,36,24,2
1,Alhue,2323,309,1,2632,0.883,0.117,0.06713,0,222,62,731,571,440,116,12,8,24,17,2
2,Alto Biobio,1269,424,1,1693,0.750,0.250,0.39734,3,80,52,923,419,201,53,10,32,68,38,2
3,Alto del Carmen,1559,241,1,1800,0.866,0.134,0.15273,1,104,68,800,476,296,123,15,6,15,24,2
4,Alto Hospicio,21589,4534,1,26123,0.826,0.174,0.07755,0,868,2070,5552,3630,4492,1743,82,54,117,76,2


In [22]:
## probabilidad apruebo que voto por piñera 
init = datetime.now()
apruebo = 1
aprobPiniera = 0
for i in values.index:
    match = general.loc[(general["Ganador"]==apruebo) & (general["ganador"]==piniera2da) & (general["categoria"]==i)]
    total = general.loc[(general["ganador"]==piniera2da) & (general["categoria"]==i)]
    if(not total.shape[0]):
        continue
    aprobPiniera += (match.shape[0]/total.shape[0]) * categ[i]
    
aprobNoPiniera = 0
## probabilidad apruebo que no voto por piñera
for i in values.index:
    match = general.loc[(general["Ganador"]==apruebo) & (general["ganador"]!=piniera2da) & (general["categoria"]==i)]
    total = general.loc[(general["ganador"]!=piniera2da) & (general["categoria"]==i)]
    if(not total.shape[0]):
        continue
    aprobNoPiniera += (match.shape[0]/total.shape[0]) * categ[i]

## probabilidad rechazo que voto por piñera 
apruebo = 0
rechazoPiniera = 0
for i in values.index:
    match = general.loc[(general["Ganador"]==apruebo) & (general["ganador"]==piniera2da) & (general["categoria"]==i)]
    total = general.loc[(general["ganador"]==piniera2da) & (general["categoria"]==i)]
    if(not total.shape[0]):
        continue
    rechazoPiniera += (match.shape[0]/total.shape[0]) * categ[i]

## probabilidad rechazo que no voto por piñera 
apruebo = 0
rechazoNoPiniera = 0
for i in values.index:
    match = general.loc[(general["Ganador"]==apruebo) & (general["ganador"]!=piniera2da) & (general["categoria"]==i)]
    total = general.loc[(general["ganador"]!=piniera2da) & (general["categoria"]==i)]
    if(not total.shape[0]):
        continue
    rechazoNoPiniera += (match.shape[0]/total.shape[0]) * categ[i]

meanPiniera = 1*aprobPiniera + 0*rechazoPiniera
meanNoPiniera = 1*aprobNoPiniera + 0*rechazoNoPiniera
print(f"Peso voto por Piñera: {meanPiniera - meanNoPiniera}")
meanPiniera = 1*aprobPiniera - 1*rechazoPiniera
meanNoPiniera = 1*aprobNoPiniera - 1*rechazoNoPiniera
print(f"Peso voto por Piñera (con -1): {meanPiniera - meanNoPiniera}")
final = datetime.now()
print(f"Tiempo de ejecución: {final-init}")

Peso voto por Piñera: -0.4115125535823638
Peso voto por Piñera (con -1): -0.3940395999183508
Tiempo de ejecución: 0:00:00.019999
